# CSPB-3104 Programming Assignment 8



1) (15 points) Implement Breadth First and Depth First Search


Given an adjacency list a,  
bfs(a, u) performs a breadth first search starting at node u and returns a list of nodes in the order in which they were seen.  
INPUT: [[1]. [2], [0]], 1  (a 3 node cycle, starting BFS at node 1)  
OUTPUT: [1, 2, 0]

dfs(a) performs a depth first search starting at node 0 and returns a list of nodes in the order in which they were seen, with start and stop times.  
INPUT: [[1], [2], [0]] (a 3 node cycle)  
OUTPUT: [(0, (1, 6)), (1, (2, 5)), (2, (3, 4))]

Note: Choose the next node in numerical order (node 3 is searched before node 5).  The adjacency lists are already sorted in this order.  
You may use the heapq library for queues.  
Be careful of the formatting for DFS.  Each element of the return list is a tuple containing an int and another tuple: (node_id, (start_time, stop_time))


[[1], [2], [0]] is the following graph: 
$$ \raisebox{.5pt}{\textcircled{\raisebox{-.9pt} {0}}}
 \\
\swarrow \;\; \nwarrow\\
\raisebox{.5pt}{\textcircled{\raisebox{-.9pt} {1}}}
 \;\rightarrow\; \raisebox{.5pt}{\textcircled{\raisebox{-.9pt} {2}}}

$$

In [1]:
#Given an adjacency list a,  
#bfs(a, u) performs a breadth first search starting at node u and returns a list of nodes in the order in which they were seen.  
#INPUT: [[1]. [2], [0]], 1  (a 3 node cycle, starting BFS at node 1)  
#OUTPUT: [1, 2, 0]

from collections import deque

def bfs(a, u):
    visited = [False] * len(a)  # initialize visited list
    queue = deque([u])  # initialize queue with the starting node
    visited[u] = True  # mark starting node as visited
    order = []  # store the order of nodes as they are visited
    
    while queue:
        node = queue.popleft()
        order.append(node)  # append current node to visited order list
        
        # traverse adjacent nodes in numerical order
        for neighbor in a[node]:
            if not visited[neighbor]:  # only visit unvisited nodes
                visited[neighbor] = True
                queue.append(neighbor)
    
    return order

In [2]:
# Given an adjacency list a,
# dfs(a) performs a depth first search starting at node 0 and returns a list of nodes in the order in which they were seen, with start and stop times.  
# INPUT: [[1], [2], [0]] (a 3 node cycle)  
# OUTPUT: [(0, (1, 6)), (1, (2, 5)), (2, (3, 4))]

def dfs(a):
    visited = [False] * len(a)  # track visited nodes
    times = []  # store nodes with their start and stop times
    time = 1  # init a global time counter

    def dfs_visit(node):
        nonlocal time
        start_time = time
        visited[node] = True
        time += 1

        # visit all unvisited neighbors in numerical order
        for neighbor in sorted(a[node]):
            if not visited[neighbor]:
                dfs_visit(neighbor)

        stop_time = time
        time += 1
        times.append((node, (start_time, stop_time)))  # append node with start and stop times

    # DFS for each component in the graph
    for i in range(len(a)):
        if not visited[i]:
            dfs_visit(i)

    # sort times by start time to ensure we maintain the expected output format 
    times.sort(key = lambda x: x[1][0])

    return times

2) (10 points) Finding cycles

Write a function that returns whether a node is part of a cycle.

HINT: Modify you DFS to return early when it finds a cycle

In [3]:
# Given an adjacency list a and an index j, returns True if node j is part of a cycle, False if not.


from typing import List, Tuple

def part_of_a_cycle(a: List[List[int]], j: int) -> bool:
    # helper function to perform DFS
    def dfs(node: int, visited: List[bool], stack: List[bool], start_stop_times: List[Tuple[int, Tuple[int, int]]], time: List[int]) -> bool:
        visited[node] = True
        stack[node] = True  # mark this node as part of the current path (recursion stack)
        time[0] += 1  # start time
        start_time = time[0]

        for neighbor in a[node]:
            if not visited[neighbor]:  # if neighbor hasnt been visited
                if dfs(neighbor, visited, stack, start_stop_times, time):
                    # if detect a cycle from the neighbor, return True
                    if neighbor == j or stack[neighbor]:
                        return True
            elif stack[neighbor]:  # found a back edge, indicating a cycle
                if neighbor == j or node == j:
                    return True
        
        stack[node] = False  # remove the node from the current path (recursion stack)
        time[0] += 1  # end time
        stop_time = time[0]
        start_stop_times.append((node, (start_time, stop_time)))
        
        return False
    
    n = len(a)
    visited = [False] * n
    stack = [False] * n  # recursion stack to detect back edges
    start_stop_times = []  # list to record nodes with start and stop times
    time = [0]  # mutable time counter
    
    # start DFS from each unvisited node
    for i in range(n):
        if not visited[i]:
            if dfs(i, visited, stack, start_stop_times, time):
                return True  # early exit if cycle found involving node j
    
    return False  # if no cycle found involving node j


Testing below

----

In [4]:
## DO NOT EDIT TESTING CODE FOR YOUR ANSWER ABOVE
# Press shift enter to test your code. Ensure that your code has been saved first by pressing shift+enter on the previous cell.
from IPython.core.display import display, HTML
def bfs_test():
    failed = False
    test_cases = [ 
        ([[1, 2, 3], [0, 2, 3], [0,1,3],[0,1,2]], 0, [0,1,2,3]),
        ([[1,3],[0],[1,3],[2]], 0, [0, 1, 3, 2]),
        ([[],[0, 2],[3],[1]], 0, [0]),
        ([[],[0, 2],[3],[1]], 1, [1, 0, 2, 3]),
        ([[1, 2], [3,4], [5,6], [7,8], [8,9], [6,7], [], [], [], []], 0, [0,1,2,3,4,5,6,7,8,9])

    ]
    for (test_graph, starting_node, solution) in test_cases:
        output = bfs(test_graph, starting_node)
        if (solution != output):
            s1 = '<font color=\"red\"> Failed - test case: Inputs: graph =' + str(test_graph) + "<br>"
            s2 = '  <b> Expected Output: </b> ' + str(solution) + ' Your code output: ' + str(output)+ "<br>"
            display(HTML(s1+s2))
            failed = True
            
    if failed:
        display(HTML('<font color="red"> One or more tests failed. </font>'))
    else:
        display(HTML('<font color="green"> All tests succeeded! </font>'))
bfs_test()

In [5]:
## DO NOT EDIT TESTING CODE FOR YOUR ANSWER ABOVE
# Press shift enter to test your code. Ensure that your code has been saved first by pressing shift+enter on the previous cell.
from IPython.core.display import display, HTML
def dfs_test():
    failed = False
    test_cases = [ 
        ([[1, 2, 3], [0, 2, 3], [0,1,3],[0,1,2]], [(0, (1, 8)), (1, (2, 7)), (2, (3, 6)), (3, (4, 5))]),
        ([[1,3],[0],[1,3],[2]], [(0, (1, 8)), (1, (2, 3)), (3, (4, 7)), (2, (5, 6))]),
        ([[],[0, 2],[3],[1]], [(0, (1, 2)), (1, (3, 8)), (2, (4, 7)), (3, (5, 6))]),
        ([[],[0, 3],[1],[]],[(0, (1, 2)), (1, (3, 6)), (3, (4, 5)), (2, (7, 8))]),
        ([[1, 2], [4,5], [3,4], [8,9], [7,8], [6,7], [], [], [], []],[(0, (1, 20)), (1, (2, 13)), (4, (3, 8)), (7, (4, 5)), (8, (6, 7)), (5, (9, 12)), (6, (10, 11)), (2, (14, 19)), (3, (15, 18)), (9, (16, 17))])

    ]
    for (test_graph, solution) in test_cases:
        output = dfs(test_graph)
        if (solution != output):
            s1 = '<font color=\"red\"> Failed - test case: Inputs: graph =' + str(test_graph) + "<br>"
            s2 = '  <b> Expected Output: </b> ' + str(solution) + ' Your code output: ' + str(output)+ "<br>"
            display(HTML(s1+s2))
            failed = True
            
    if failed:
        display(HTML('<font color="red"> One or more tests failed. </font>'))
    else:
        display(HTML('<font color="green"> All tests succeeded! </font>'))
dfs_test()

In [6]:
## DO NOT EDIT TESTING CODE FOR YOUR ANSWER ABOVE
# Press shift enter to test your code. Ensure that your code has been saved first by pressing shift+enter on the previous cell.
from IPython.core.display import display, HTML
def part_of_a_cycle_test():
    failed = False
    test_cases = [ 
        ([[1, 2, 3], [0, 2, 3], [0,1,3],[0,1,2]], 0, True),
        ([[1,3],[],[1,3],[2]], 0, False),
        ([[1,3],[],[1,3],[2]], 2, True),
        ([[],[0, 2],[3],[1]], 0, False),
        ([[],[0, 2],[3],[1]], 1, True),
        ([[1, 2], [4,5], [3,4], [8,9], [7,8], [6,7], [], [], [], []], 0, False)

    ]
    for (test_graph, starting_node, solution) in test_cases:
        output = part_of_a_cycle(test_graph, starting_node)
        if (solution != output):
            s1 = '<font color=\"red\"> Failed - test case: Inputs: graph =' + str(test_graph) + ' node ' + str(starting_node) + "<br>"
            s2 = '  <b> Expected Output: </b> ' + str(solution) + ' Your code output: ' + str(output)+ "<br>"
            display(HTML(s1+s2))
            failed = True
            
    if failed:
        display(HTML('<font color="red"> One or more tests failed. </font>'))
    else:
        display(HTML('<font color="green"> All tests succeeded! </font>'))
part_of_a_cycle_test()